In [14]:
#Imports
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from scipy.interpolate import make_interp_spline
import numpy as np
from matplotlib.collections import LineCollection
import sqlite3
pd.set_option('display.max_columns', None)

import os

## Loading Datasets
Note ; If name already exists in file, add extension _{digit} Example -> etsy-sold-orders.csv becomes etsy-sold-orders_2.csv
####    Etsy dataset names
````
- etsy-sold-orders.csv
- etsy-deposits.csv
- etsy-sold-order-items.csv
- etsy-direct-checkout-payments.csv
````
    
####    Yoycol dataset names
````
- yoycol-order-details.csv
````
####    Printify dataset names
````
- printify-orders.csv
- printify-orders_2.csv
 ````

In [15]:
file_ext = 'updData'

etsy_orders = pd.read_csv(f'{file_ext}/etsy-sold-orders.csv')
etsy_deposits = pd.read_csv(f'{file_ext}/etsy-deposits.csv')
etsy_items = pd.read_csv(f'{file_ext}/etsy-sold-order-items.csv')
etsy_payments = pd.read_csv(f'{file_ext}/etsy-direct-checkout-payments.csv')

yoycol_orders = pd.read_csv(f'{file_ext}/yoycol-order-details.csv')

printify_orders = pd.read_csv(f'{file_ext}/printify-orders.csv')
printify_orders_2 = pd.read_csv(f'{file_ext}/printify-orders_2.csv')
printify_orders_3 = pd.read_csv(f'{file_ext}/printify_orders_3.csv')
printify_orders_4 = pd.read_csv(f'{file_ext}/printify_orders_4.csv')

# Connecting to Sqlite3 Database

#### Creating Database and connecting to it

In [94]:
connection = sqlite3.connect('DashboardDataHub.db')

In [95]:
cursor = connection.cursor()

#### Creating tables and populating them

In [101]:
etsy_orders.to_sql('etsy_orders', connection, if_exists='replace')
etsy_deposits.to_sql('etsy_deposits', connection, if_exists='replace')
etsy_items.to_sql('etsy_items', connection, if_exists='replace')
etsy_payments.to_sql('etsy_payments', connection, if_exists='replace')
yoycol_orders.to_sql('yoycol_orders', connection, if_exists='replace')
printify_orders.to_sql('printify_orders', connection, if_exists='replace')
printify_orders2.to_sql('printify_orders2', connection, if_exists='replace')

13

#### Validating from DB

In [125]:
# cursor.execute("Select * From etsy_orders Where ShipState = TX")
cursor.execute("PRAGMA table_info(etsy_orders);")
cursor.fetchall()[:5]

#cursor.execute("Select * From etsy_orders Where [Ship State] like 'TX';")
#cursor.fetchall()

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'Sale Date', 'TEXT', 0, None, 0),
 (2, 'Order ID', 'INTEGER', 0, None, 0),
 (3, 'Buyer User ID', 'TEXT', 0, None, 0),
 (4, 'Full Name', 'TEXT', 0, None, 0)]

In [126]:
connection.commit()

In [98]:
df_orders = pd.read_sql_query('Select * from etsy_orders', connection)

## Data and Functions for Orders View

#### Combining Etsy's data

In [16]:

# Joining 2  Dataset -> Orders and Items, by OrderID
items_cols_to_join = ['Order ID', 'Ship Name', 'Ship Address1', 'Ship Address2', 'Variations', 'Date Paid', 'Quantity', 'Item Total', 'Item Name', 'Transaction ID', 'Price']

items_to_join = etsy_items[items_cols_to_join]
items_to_join.columns = ['Order ID', 'Item-Ship Name', 'Item-Ship Address1', 'Item-Ship Address2', 'Item-Variations', 'Item-Date Paid', 'Item-Quantity', 'Item-Item Total', 'Item-Item Name', 'Item-Transaction ID', 'Item-Price']

orders_items_joined = etsy_orders.copy()
orders_items_joined = pd.merge(orders_items_joined,items_to_join,on='Order ID',how='outer')

# drop_duplicates()

#### Integrating distributors data

In [17]:
def ordersDataTransform(**kwargs):
    # Combining all datasets of the same kind that come in multiples
    stores_ID_keys = {'etsy':"Order ID", 'printify': "Sales channel ID", 'yoycol':"Store order ID"}
    data_holder = {}
    for source, datasets in kwargs.items():
        order_frames = None
        if not isinstance(datasets, list):
            raise IndexError('InvalidIndexError -> dataset passed must be of type list including singular')
        if len(datasets) > 1:
            for i in range(len(datasets)):
                if order_frames is None:
                    order_frames = pd.DataFrame(datasets[0])
                    if i + 1 < len(datasets):
                        temp_frames = pd.concat([order_frames, pd.DataFrame(datasets[i+1])])
                        if order_frames.duplicated().any():
                            order_frames.drop_duplicates(inplace=True)
                        else:
                            order_frames = temp_frames
                    else:
                        data_holder[source] = order_frames
                else:
                    if i + 1 < len(datasets):
                        temp_frames = pd.concat([order_frames, pd.DataFrame(datasets[i+1])])
                        if order_frames.duplicated().any():
                            order_frames.drop_duplicates(inplace=True)
                        else:
                            order_frames = temp_frames
                    else:
                        data_holder[source] = order_frames
        else:
            data_holder[source] = pd.DataFrame(datasets[0])

    iter_index = 0
    on_key = 'Order ID'
    # Bring all datasets together for the final Orders_all dataset
    for src, data in data_holder.items():
        mapper_key = stores_ID_keys.get(src.lower())
        if mapper_key != on_key:
            data[on_key] = data[mapper_key]
            data.drop(columns=[mapper_key], inplace=True)
            data.columns = data.columns.map(lambda x: src[:4] + "-" + x if x != on_key else x)
            
            
        data.set_index('Order ID', inplace=True)
        data.index = data.index.map(lambda x:str(x))
        if iter_index == 0:
            left_data = data
        else:
            left_data = left_data.join(data)
        iter_index += 1
    return left_data

In [18]:
orders_all = ordersDataTransform(etsy= [orders_items_joined], printify=[printify_orders, printify_orders_2, printify_orders_3, printify_orders_4], yoycol=[yoycol_orders])

In [19]:
orders = orders_all.copy()


In [775]:

query = orders.index == '2581208238'
query.any()

False

In [747]:
orders_all[orders_all['prin-Date created'].isnull()]['yoyc-Created at']

Order ID
2581208238                    NaN
2588129685    2022-09-07 12:09:37
2588971167    2022-09-08 08:09:14
2597567764    2022-09-13 09:22:29
2601131957    2022-09-20 03:51:20
2601327225    2022-09-20 00:10:29
2603597361    2022-09-22 02:10:14
2605367029    2022-09-23 18:10:58
2606290199    2022-09-24 13:16:40
2612073269    2022-09-30 02:10:44
2626879413    2022-10-14 04:09:27
2629335152    2022-10-12 00:10:58
2630109082    2022-10-12 14:10:03
2646742964    2022-10-27 06:09:22
2647245525    2022-11-01 10:11:15
2648859981    2022-11-03 00:09:20
2652711442    2022-11-01 16:01:40
2654190407    2022-11-07 02:09:49
2656110524    2022-11-04 08:09:24
Name: yoyc-Created at, dtype: object

In [ ]:


orders['Image Path'] = np.nan

In [778]:
df.index.map(lambda x: x[:5])

Index(['25881', '25889', '25975', '25994', '26011', '26013', '26015', '26018',
       '26035', '26053', '26062', '26062', '26064', '26084', '26084', '26089',
       '26092', '26098', '26101', '26102', '26120', '26128', '26128', '26130',
       '26133', '26135', '26142', '26142', '26147', '26149', '26158', '26167',
       '26177', '26181', '26188', '26196', '26208', '26221', '26221', '26232',
       '26248', '26251', '26258', '26261', '26268', '26273', '26274', '26278',
       '26287', '26293', '26295', '26301', '26305', '26318', '26408', '26408',
       '26410', '26419', '26421', '26435', '26440', '26441', '26446', '26453',
       '26455', '26455', '26463', '26463', '26466', '26467', '26467', '26472',
       '26474', '26488', '26499', '26502', '26512', '26514', '26517', '26525',
       '26527', '26527', '26533', '26539', '26539', '26541', '26559', '26561',
       '26567', '26593', '26602', '26606'],
      dtype='object', name='Order ID')

In [20]:
df = orders_all.copy()


#### Timeframe resample
- Daily
- Weekly
- Monthly
- Yearly


#### Order By
- Date: Most recent
- Date: Older First
- Price: Highest to lowest
- Price: Lowest to highest
- Most Sold
- **maybe** Yields Best Profits


In [945]:
import os
import re

In [1171]:
s = []
for i in df.index:
   s.append(i[5:])

In [1172]:
len(pd.Series(s).unique())

90

In [1173]:
len(s)

93

In [1178]:
df['Image Path'][df['Image Path'].index == '2581208238'].values[0]

'media/no-prod-img'

In [1180]:
df['Image Path'].unique()

array(['media/no-prod-img',
       'Version 2 of Going Merry ! One Piece Spiral Bound Notebook Journal.jpg'],
      dtype=object)

In [1191]:
df['Sale Date'][2].strftime('%y-%b-%d')

'22-Sep-07'

In [1227]:
df['Progress'] = df.apply(lambda x: 'Closed', axis=1)

In [1220]:
df['Source'] = df.apply(lambda x: 'Printify' if not np.isnan(x['prin-Printify ID']) else 'Yoycol', axis=1)

In [ ]:
colors = {'Closed' : 'light blue', 'Open': 'Green', 'On Transit': 'Orange', 'Canceled': 'Red'}
orders_data['Progress Color'] = orders_data.apply(lambda x: colors.get(x),  axis=1)

In [ ]:
df.iloc[2]['prin-Printify ID'].isnan

In [1217]:
np.isnan(df.iloc[2]['prin-Printify ID'])

True

In [1240]:
df['Progress'][df['Progress'].index == '2581208238'] = 'Test'

/var/folders/y0/lrdnkktn1v17ttgx4l56mdzw0000gn/T/ipykernel_6960/1656262694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Progress'][df['Progress'].index == '2581208238'] = 'Test'


In [ ]:
df.index == '2581208238'

In [1255]:
df['Progress'][df['Progress'].index == '2588129685'] = 'Test'

/var/folders/y0/lrdnkktn1v17ttgx4l56mdzw0000gn/T/ipykernel_6960/1663960511.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Progress'][df['Progress'].index == '2588129685'] = 'Test'


In [1259]:
df['Progress'][df['Progress'].index[3]] = 'Yes'

/var/folders/y0/lrdnkktn1v17ttgx4l56mdzw0000gn/T/ipykernel_6960/296491439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Progress'][df['Progress'].index[3]] = 'Yes'


In [49]:
eval_progress = {'2581208238': 'Open'}

In [50]:
df['Progress'] = df.index.map(lambda x: eval_progress.get(x) if eval_progress.get(x) else 'Closed')

In [ ]:
df.head(2)

In [ ]:
df['Sale Date'].loc[today - show_date: today]

In [85]:
df['Sale Date']= df['Sale Date'].apply(lambda x: pd.to_datetime(x))
df['Sale Date']

Order ID
2581208238   2022-08-28
2588129685   2022-09-06
2588971167   2022-09-07
2597567764   2022-09-12
2599495408   2022-09-14
                ...    
2656110524   2022-11-03
2656774348   2022-11-04
2659301066   2022-11-06
2660261866   2022-11-06
2660614142   2022-11-06
Name: Sale Date, Length: 103, dtype: datetime64[ns]

In [73]:
pd.to_datetime(today-show_date)

Timestamp('2022-10-11 01:07:48.948186')

In [ ]:
df['Sale Date'].loc[:today]